### Import Library

In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import gdown
import joblib

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import lightgbm as lgb

import eli5
from eli5.sklearn import PermutationImportance

# 코드 셀 실행 후 경고를 무시
# import warnings
# warnings.filterwarnings(action='ignore')

### Data Load

In [2]:
# base path 만 고쳐서 한 번에 불러오세요
base_path = '../../data/'

train_path = base_path + 'train.csv'
test_path  = base_path + 'test.csv'
dt = pd.read_csv(base_path + 'train.csv')
dt_test = pd.read_csv(base_path + 'test.csv')
va1 = pd.read_csv(base_path + '2023년_공동주택_공시가격_정보.csv', encoding='cp949')

## Feature Engineering

### Train & Test set 데이터 처리

In [3]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
dt['is_test'] = 0
dt_test['is_test'] = 1
df = pd.concat([dt, dt_test])

In [4]:
# 필요 없어 보이는 columns 제거
drop_col = ['부번', '계약일', 'k-전화번호', 'k-팩스번호', 'k-관리방식', 'k-복도유형', 'k-시행사', 'k-사용검사일-사용승인일', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일', 'k-관리비부과면적', '주차대수', '건축면적', '해제사유발생일', '단지소개기존clob', 'k-135㎡초과', '중개사소재지', '등기신청일자', '거래유형', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-단지분류(아파트,주상복합등등)', 'k-세대타입(분양형태)', 'k-난방방식', 'k-건설사(시공사)']
df.drop(drop_col, axis=1, inplace=True)

In [5]:
# 시군구 feaeture 처리
# 구와 동에 대한 Feature 수정
df['구'] = df['시군구'].apply(lambda x:x.split()[1])
df['동'] = df['시군구'].apply(lambda x:x.split()[2])
omg = ['용산구', '강남구', '서초구', '송파구', '성동구', '종로구']
is_omg = []
for x in df['구'].tolist():
    if x in omg:
        is_omg.append(1)
    else:
        is_omg.append(0)

df.loc[~df['구'].isin(omg), '동'] = 'Unknown' 

del df['시군구']

# 본번, 부번의 경우 float로 되어있지만 범주형 변수의 의미를 가지므로 object(string) 형태로 바꾸어주고 아래 작업을 진행하겠습니다.
df['본번'] = df['본번'].astype('str')

# 계약 년도 데이터 생성
df['계약년'] = df['계약년월'].astype('str').map(lambda x : x[:4])
df['계약월'] = df['계약년월'].astype('str').map(lambda x : x[4:])

# 먼저, 연속형 변수와 범주형 변수를 위 info에 따라 분리해주겠습니다.
continuous_columns = []
categorical_columns = []

for column in df.columns:
    if pd.api.types.is_numeric_dtype(df[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

# 범주형 변수에 대한 보간
df[categorical_columns] = df[categorical_columns].fillna('NULL')

연속형 변수: ['전용면적(㎡)', '계약년월', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test']
범주형 변수: ['번지', '본번', '아파트명', '도로명', '구', '동', '계약년', '계약월']


### 공시가격 데이터 처리

In [6]:
va1['도로명주소']= va1['도로명주소'].apply(lambda address: address.split()[2]+' '+address.split()[-1] )
va1.rename(columns={'도로명주소': '도로명',
                     '단지명':'아파트명'}, inplace=True)

va1 = va1[va1['시도']=='서울특별시']
va1 = va1.drop(columns={'기준월','법정동코드','읍면','특수지코드','특수지명','단지코드','동코드','호코드',})
va1['공시가격']=va1['공시가격']/10000

In [7]:
avg_price_by_road = va1.groupby('도로명')['공시가격'].mean().reset_index()
avg_price_by_road.columns = ['도로명', '평균공시가격']  # 열 이름 변경

df = pd.merge(df, avg_price_by_road, on='도로명', how='left')

In [8]:
grouped_prices = va1.groupby('도로명')['공시가격'].apply(list).reset_index()
grouped_prices.columns = ['도로명', '공시가격리스트']  # 열 이름 변경

# 기존 데이터프레임에 공시가격리스트 열 추가
df = pd.merge(df, grouped_prices, on='도로명', how='left')

In [13]:
df.loc[df['아파트명']=='갤러리아포레', :]['공시가격리스트']

62290      [316300.0, 405300.0, 343100.0, 318500.0, 41000...
62291      [316300.0, 405300.0, 343100.0, 318500.0, 41000...
62292      [316300.0, 405300.0, 343100.0, 318500.0, 41000...
62293      [316300.0, 405300.0, 343100.0, 318500.0, 41000...
62294      [316300.0, 405300.0, 343100.0, 318500.0, 41000...
                                 ...                        
886980                                                   NaN
886981                                                   NaN
886982                                                   NaN
1124863    [316300.0, 405300.0, 343100.0, 318500.0, 41000...
1124864    [316300.0, 405300.0, 343100.0, 318500.0, 41000...
Name: 공시가격리스트, Length: 204, dtype: object

In [ ]:
df['평균공시가격']=df['평균공시가격'].interpolate(method='linear', axis=0)

### 좌표데이터 처리

In [24]:
# pickle에서 데이터프레임 다시 불러오기
with open('df.pickle', 'rb') as f:
    df_loaded = pickle.load(f)

In [28]:
# df['공시가격리스트']= df['공시가격리스트'].astype(str)
df.drop(['공시가격리스트'], axis=1, inplace=True)
df['평균공시가격']=df['평균공시가격'].interpolate(method='linear', axis=0)

### Train

In [27]:
df_train = df.loc[df['is_test']==0, :]
df_test = df.loc[df['is_test']==1, :]

df_train.drop(['is_test'], axis=1, inplace=True)
df_test.drop(['is_test'], axis=1, inplace=True)
print(df_train.shape, df_test.shape)

(1118822, 17) (9272, 17)


In [2]:
import os
os.getcwd()

'/root/upstage-ml-regression-3/baseline'